In [15]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras import models
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.13.0
sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)
matplotlib 3.7.2
numpy 1.24.3
pandas 2.0.3
sklearn 1.3.0
tensorflow 2.13.0


AttributeError: module 'keras.api._v2.keras' has no attribute '__version__'

In [16]:
y_true = [[0., 1.], [0., 0.]]
y_pred = [[1., 1.], [1., 0.]]
# Using 'auto'/'sum_over_batch_size' reduction type.
mse = tf.keras.losses.MeanSquaredError()
mse(y_true, y_pred)


<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [17]:
m = tf.keras.metrics.MeanSquaredError()  #具有累计效果
m.update_state([[0, 1], [0, 0]], [[1, 1], [1, 0]])
m.result().numpy()

0.5

In [18]:
#在原有的基础上继续计算损失
y_true = [[0., 1.], [0., 0.]]
y_pred = [[2., 1.], [2., 0.]]
# m.reset_states()
m.update_state(y_true,y_pred)
m.result().numpy()

1.25

In [19]:
y_true = [[0., 1.], [0., 0.]]
y_pred = [[2., 1.], [2., 0.]]
mse = tf.keras.losses.MeanSquaredError()
mse(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [20]:
y_true = tf.constant([[0.], [0.]])
y_pred = tf.constant([[2.], [1.]])
ret=tf.square(y_pred - y_true)  #传入的y_pred - y_true必须是tensor
ret

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[4.],
       [1.]], dtype=float32)>

In [21]:
tf.reduce_mean()

TypeError: Missing required positional argument

In [36]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing(data_home='data')
# print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

(20640, 8)
(20640,)


In [37]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)


(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [38]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [39]:
#自定义损失函数
def customized_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))
# m=tf.keras.losses.MeanSquaredError()  #用框架提供的对象
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])
model.summary()
model.compile(loss=customized_mse, optimizer="sgd",
              metrics=["mean_squared_error"])  #为了验证自定义损失是否正确
callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                270       
                                                                 
 dense_1 (Dense)             (None, 1)                 31        
                                                                 
Total params: 301 (1.18 KB)
Trainable params: 301 (1.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
